In [1]:
# Set-Ups 
from flask import Flask, request, jsonify, send_file, render_template
from werkzeug.utils import secure_filename
import joblib
import numpy as np
import pandas as pd

import sqlalchemy
from sqlalchemy import create_engine, inspect, select, text, and_, or_, func
from sqlalchemy import Table, MetaData

from datetime import datetime
import calendar

import json
import matplotlib.pyplot as plt

In [11]:
# Connect to database
# If error pop-up as "No module called psycopg2", try install psycopg2 first
# pip install psycopg2
passwd = '200101'
w_engine = create_engine('postgresql+psycopg2://postgres:' + passwd + '@localhost:5432/Proj-App', echo=True)

m2 = MetaData(bind=w_engine)
m2.reflect()


2022-03-24 01:55:50,161 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-03-24 01:55:50,162 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-24 01:55:50,162 INFO sqlalchemy.engine.Engine select current_schema()
2022-03-24 01:55:50,163 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-24 01:55:50,163 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-03-24 01:55:50,164 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-24 01:55:50,165 INFO sqlalchemy.engine.Engine SELECT c.relname FROM pg_class c JOIN pg_namespace n ON n.oid = c.relnamespace WHERE n.nspname = %(schema)s AND c.relkind in ('r', 'p')
2022-03-24 01:55:50,165 INFO sqlalchemy.engine.Engine [generated in 0.00055s] {'schema': 'public'}
2022-03-24 01:55:50,169 INFO sqlalchemy.engine.Engine 
            SELECT c.oid
            FROM pg_catalog.pg_class c
            LEFT JOIN pg_catalog.pg_namespace n ON n.oid = c.relnamespace
            WHERE (pg_catalog.pg_table_is_visible(c.oid))
            

In [12]:
# Test connection
for tt in m2.tables:
    print(tt)

rent_history
user_info
house_info
area_info


Func 1: Return Lowest Price House to Highest

In [13]:
house_info = Table("house_info", m2, autoload=True)

# Try the query
stmt1 = select([house_info]).where(house_info.c.house_status == 'FOR RENT').\
    order_by(house_info.c.expected_price).limit(2)

with w_engine.connect() as con:
        d1 = con.execute(stmt1)

d1.fetchall()


2022-03-24 02:05:09,136 INFO sqlalchemy.engine.Engine SELECT house_info.house_title, house_info.area, house_info.room_size, house_info.house_location, house_info.postal_code, house_info.number_of_bedrooms, house_info.number_of_washrooms, house_info.max_tenant, house_info.available_date, house_info.expected_price, house_info.price_per_feet, house_info.negotiable, house_info.owner_email, house_info.owner_phone_number, house_info.house_status 
FROM house_info 
WHERE house_info.house_status = %(house_status_1)s ORDER BY house_info.expected_price 
 LIMIT %(param_1)s
2022-03-24 02:05:09,137 INFO sqlalchemy.engine.Engine [generated in 0.00119s] {'house_status_1': 'FOR RENT', 'param_1': 2}


[('Marina Two Residences', 'Boat Quay', 554, '29 Marina Way', 18950, 3, 3, 6, datetime.date(2022, 3, 30), Decimal('3500.00'), Decimal('6.31'), 'YES', 'hkohlert58@cloudflare.com', '2446353960', 'FOR RENT'),
 ('The Sail @ Marina Bay', 'Boat Quay', 592, '2 Marina Boulevard', 18987, 1, 1, 3, datetime.date(2022, 4, 18), Decimal('5000.00'), Decimal('8.45'), 'YES', 'snapolione2u@ebay.co.uk', '3095436558', 'FOR RENT')]

Add filters to query

In [22]:
# Filters in WHERE clause
Area = 'Boat Quay'
Negotiable = 'YES'
Available_date = '2022-3-25'
Bedroom_no = 1

# Ranking : expected_price, price_per_feet, room_size
# Variable : ASC or DESC
ranking = 'expected_price'

# Try the query
A_date = datetime.strptime(Available_date, '%Y-%m-%d')
stmt2 = select([house_info]).where(house_info.c.house_status == 'FOR RENT').\
    where(house_info.c.area == Area).\
    where(house_info.c.negotiable == Negotiable).\
    where(house_info.c.available_date < A_date).\
    where(house_info.c.number_of_bedrooms == Bedroom_no).\
    order_by(house_info.c.expected_price).limit(2)

with w_engine.connect() as con:
        d2 = con.execute(stmt2)

d2.fetchall()


2022-03-24 02:39:02,287 INFO sqlalchemy.engine.Engine SELECT house_info.house_title, house_info.area, house_info.room_size, house_info.house_location, house_info.postal_code, house_info.number_of_bedrooms, house_info.number_of_washrooms, house_info.max_tenant, house_info.available_date, house_info.expected_price, house_info.price_per_feet, house_info.negotiable, house_info.owner_email, house_info.owner_phone_number, house_info.house_status 
FROM house_info 
WHERE house_info.house_status = %(house_status_1)s AND house_info.area = %(area_1)s AND house_info.negotiable = %(negotiable_1)s AND house_info.available_date < %(available_date_1)s AND house_info.number_of_bedrooms = %(number_of_bedrooms_1)s ORDER BY house_info.expected_price 
 LIMIT %(param_1)s
2022-03-24 02:39:02,288 INFO sqlalchemy.engine.Engine [cached since 866.8s ago] {'house_status_1': 'FOR RENT', 'area_1': 'Boat Quay', 'negotiable_1': 'YES', 'available_date_1': datetime.datetime(2022, 3, 25, 0, 0), 'number_of_bedrooms_1': 1

[('The Clift', 'Boat Quay', 764, '21 McCallum Street', '069047', 1, 1, 3, datetime.date(2022, 3, 24), Decimal('5300.00'), Decimal('6.94'), 'YES', 'cfreestone3a@uiuc.edu', '2239272334', 'FOR RENT')]

In [21]:
def ex_asc_f(Area, Negotiable, Available_date, Bedroom_no):
    A_date = datetime.strptime(Available_date, '%Y-%m-%d')
    stmt = select([house_info]).where(house_info.c.house_status == 'FOR RENT').\
        where(house_info.c.area == Area).\
        where(house_info.c.negotiable == Negotiable).\
        where(house_info.c.available_date < A_date).\
        where(house_info.c.number_of_bedrooms == Bedroom_no).\
        order_by(house_info.c.expected_price).limit(2)
    
    with w_engine.connect() as con:
        d = con.execute(stmt)
    
    return d.fetchall()

In [20]:
app = Flask(__name__)

@app.route("/expected_price/asc")
def ex_asc():
    Area = request.args.get('Area') 
    Negotiable = request.args.get('Negotiable')
    Available_date = request.args.get('Available_date')
    Bedroom_no = request.args.get('Bedroom_no')

    s1 = ex_asc_f(Area, Negotiable, Available_date, Bedroom_no)

    c1,c2,c3,c4,c5,c6,c7,c8 = [],[],[],[],[],[],[],[]
    c9,c10,c11,c12,c13,c14,c15 = [],[],[],[],[],[],[]

    for i in range(len(s1)):
        c1.append(s1[i][0])
        c2.append(s1[i][1])
        c3.append(s1[i][2])
        c4.append(s1[i][3])
        c5.append(s1[i][4])
        c6.append(s1[i][5])
        c7.append(s1[i][6])
        c8.append(s1[i][7])   
        c9.append(s1[i][8])  
        c10.append(s1[i][9])  
        c11.append(s1[i][10])  
        c12.append(s1[i][11])  
        c13.append(s1[i][12])  
        c14.append(s1[i][13])  
        c15.append(s1[i][14]) 
    
    dic = {"house_title":c1,"area":c2,"room_size":c3,"house_location":c4,
    "postal_code":c5,"number_of_bedrooms":c6,"number_of_washrooms":c7,
    "max_tenant":c8,"available_date":c9,"expected_price":c10,"price_per_feet":c11,
    "negotiable":c12,"owner_email":c13,"owner_phone_number":c14,"house_status":c15}

    return dic



